In [1]:
import numpy as np
import tensorflow as tf
import keras
from numba import cuda
import time

# Utils Function

# Convolution

In [5]:

class Layer():
  def forward(self,inputs):
    pass
  def backward(self,output_gradient, learning_rate):
    pass

@cuda.jit
def conv_forward_kernel(input,weights,stride, output):
  n_chanels=input.shape[2]
  n_filters,filter_size,_,__=weights.shape
  output_height,output_width,_ = output.shape
  row,col,fillterIdx = cuda.grid(3)
  if(row>=output_height or col>=output_width or fillterIdx>=n_filters):
    return
  sum=0
  for chanel_idx in range(n_chanels):
      for fillterRow in range(filter_size):
        for fillterCol in range(filter_size):
            iR = row*stride + fillterRow
            iC = col*stride + fillterCol
            sum+=input[iR,iC,chanel_idx]*weights[fillterIdx,fillterRow,fillterCol,chanel_idx]
  output[ row, col,fillterIdx] = sum
  

@cuda.jit
def conv_backward_kernel(input,weights,stride, output):
  n_chanels=input.shape[-1]
  n_filters,filter_size,_,__=weights.shape
  output_height,output_width,_ = output.shape
  row,col,fillterIdx = cuda.grid(3)
  if(row>=output_height or col>=output_width or fillterIdx>=n_filters):
    return
  sum=0
  for chanel_idx in range(n_chanels):
      for fillterRow in range(filter_size):
        for fillterCol in range(filter_size):
            iR = row*stride + fillterRow
            iC = col*stride + fillterCol
            input_gradient[ iR, iC,i_chanel] += (
                  self.weights[fillterIdx,iR,iC,i_chanel] * output_gradient[ row, col,filter_index])
            filter_gradient[fillterIdx,iR,iC,i_chanel] += (
                  self.inputs[ iR, iC,i_chanel] * output_gradient[row, col,fillterIdx])
  output[ row, col,fillterIdx] = sum
  pass

class Convolution(Layer):
  def __init__(self,n_filters=32,filter_size=3 ,stride=1,activation=None,input_shape=(28,28,1)):
      self.n_filters = n_filters
      self.filter_size = filter_size
      self.stride = stride
      self.activation = activation
      np.random.seed(10)
      # self.filters = np.random.randn(n_filters, filter_size, filter_size) / (filter_size * filter_size)
      self.weights = np.random.randn(n_filters,filter_size, filter_size,input_shape[2])/  np.sqrt(2 / (input_shape[2] * filter_size * filter_size))
      self.bias = np.zeros((n_filters, 1))
      
      self.inputs=None
      self.use_device=False
  def forward(self,inputs):
      self.inputs =inputs
      n_batchs, in_height,in_width,n_chanels=inputs.shape
      output_width = int((in_width - self.filter_size ) / self.stride) + 1
      output_height = int((in_height - self.filter_size ) / self.stride) + 1
      outputs = np.zeros( (n_batchs,output_height,output_width,  self.n_filters))
      start= time.time()
      ## ===========USING CPU===========##
      if(self.use_device==False):
        for i_idx in range(n_batchs):
          for row in range(output_height):
            for col in range(output_width):
              for f_idx in range(self.n_filters):
                        row_start = row * self.stride
                        row_end = row_start + self.filter_size
                        col_start = col * self.stride
                        col_end = col_start + self.filter_size
                        sum=0
                        for in_chanel in range(self.weights.shape[-1]):
                          sum+= np.sum( self.weights[f_idx,:,:,in_chanel]* inputs[i_idx,  row_start:row_end, col_start:col_end,in_chanel])
                        outputs[i_idx, row, col,f_idx] =sum
      ## ===========USING DEVICE===========##
      else:
        block_size = (4,4,4)
        grid_size = ((output_height-1)//block_size[0]+1,(output_width-1)//block_size[1]+1,(self.n_filters-1)//block_size[2]+1)
        out_device= cuda.to_device(outputs[0])

        weights_device = cuda.to_device(self.weights)
        for i in range(n_batchs):
          inputs_device = cuda.to_device(inputs[i])
          conv_forward_kernel[grid_size,block_size](inputs_device,weights_device,1,out_device)
          outputs[i]=out_device.copy_to_host()
      ## ===========END USING DEVICE===========##
      if(self.activation=="relu"):
        outputs = np.maximum(0,outputs)
      end= time.time()
      print("time:",end-start )
      return outputs
  def backward(self, output_gradient, learning_rate):
      n_batchs, input_height, input_width,input_channels = self.inputs.shape
      _,  output_height, output_width,num_filters = output_gradient.shape
      filter_gradient = np.zeros(self.weights.shape)
      input_gradient =  np.zeros(self.inputs.shape)

      for image_index in range(n_batchs):
          for filter_index in range(num_filters):
              for row in range(output_height):
                  for col in range(output_width):
                      row_start = row * self.stride
                      row_end = row_start + self.filter_size
                      col_start = col * self.stride
                      col_end = col_start + self.filter_size
                      for i_chanel in range(inputs.shape[-1]):
                        input_gradient[image_index,  row_start:row_end, col_start:col_end,i_chanel] += (
                              self.weights[filter_index,:,:,i_chanel] * output_gradient[image_index, row, col,filter_index])
                        filter_gradient[filter_index,:,:,i_chanel] += (
                              self.inputs[image_index, row_start:row_end, col_start:col_end,i_chanel] * output_gradient[
                          image_index, row, col,filter_index])

      self.weights -= learning_rate * filter_gradient/n_batchs
      if(self.activation=="relu"):
        input_gradient[self.inputs <= 0] = 0
      return input_gradient



In [4]:
inputs=np.random.randint(1,100, (10,28,28,1))
conv =Convolution(n_filters=32, filter_size=3, stride=1,activation='relu',input_shape=(28,28,1))
outputs_host=conv.forward(inputs)
conv.use_device=True
outputs_device=conv.forward(inputs)
# np.sum(np.abs(outputs_device-outputs_host))

input_grad=conv.backward(outputs_device,0.01)

NameError: ignored

AttributeError: ignored

In [ ]:

conv.use_device=True
outputs_device=conv.forward(inputs)
conv.use_device=False
outputs_host=conv.forward(inputs)


time excute: 1.2260425090789795
time excute: 15.554473161697388


In [ ]:
print("loss=",np.sum(np.abs(outputs_host-outputs_device)))

loss= 1.863692968072199e-09


### GPU

# Max Pooling


In [ ]:
@cuda.jit
def maxPool2D_forward_kernel(inputs, outputs,stride,pool_size):
  n_batchs,in_height,in_width,n_chanels=inputs.shape
  n_batchs,output_height,output_width,_ = outputs.shape
  ibatch,h,w = cuda.grid(3)
          # Max pool over input
  if(ibatch>=n_batchs or h>=output_height or w>=output_width): return
  h_start = h * stride
  h_end = h_start + pool_size
  w_start = w * stride
  w_end = w_start + pool_size
  for i_chanel in range(n_chanels):
    # findmax
    slice_input=inputs[ibatch,h_start:h_end, w_start:w_end,i_chanel]
    max_value=slice_input[0,0]
    for h_pool in range(pool_size):
      for w_pool in range(pool_size):
        max_value=max(max_value,slice_input[h_pool,w_pool])

    outputs[ibatch,h, w,i_chanel] =max_value


class MaxPool2D(Layer):
    def __init__(self, pool_size=2, stride=2):
        self.pool_size = pool_size
        self.stride = stride
        self.use_device=False
        self.inputs=None
    def forward(self, inputs):
        # Save input
        self.inputs = inputs
        batch_size, input_height, input_width,num_channels = inputs.shape
        output_height = int((input_height - self.pool_size) / self.stride) + 1
        output_width = int((input_width - self.pool_size) / self.stride) + 1
        
        # Initialize output
        start= time.time()
        ## ===========USING CPU===========##

        outputs =np.zeros((batch_size, output_height, output_width,num_channels))
        if(self.use_device==False):
          # Max pool over input
          for h in range(output_height):
              for w in range(output_width):
                  h_start = h * self.stride
                  h_end = h_start + self.pool_size
                  w_start = w * self.stride
                  w_end = w_start + self.pool_size
                  outputs[:,h, w,:] = np.max(inputs[:, h_start:h_end, w_start:w_end,:], axis=(1, 2))
        ## ===========USING DEVICE===========##
        else:
          out_device= cuda.device_array_like(outputs)
          block_size = (8,4,4)
          grid_size = ((batch_size-1)//block_size[0]+1,(output_height-1)//block_size[1]+1,(output_width-1)//block_size[2]+1)
          inputs_device = cuda.to_device(inputs)
          maxPool2D_forward_kernel[grid_size,block_size](inputs_device,out_device,self.stride,self.pool_size)
          outputs=out_device.copy_to_host()
      ## ===========END USING DEVICE===========##
        end = time.time()
        print('time=',end-start)
        return outputs
    
    def backward(self, output_gradient, learning_rate):
        # Initialize input error
        input_gradient = np.zeros(self.inputs.shape)
        batch_size, output_height, output_width,num_channels = output_gradient.shape
        # Max pool over input gradients
        for h in range(output_height):
            for w in range(output_width):
                h_start = h * self.stride
                h_end = h_start + self.pool_size
                w_start = w * self.stride
                w_end = w_start + self.pool_size
                input_slice = self.inputs[:, h_start:h_end, w_start:w_end,:]
                max_vals = np.max(input_slice, axis=(1, 2), keepdims=True)
                max_mask = (input_slice == max_vals)
                input_gradient[:,h_start:h_end, w_start:w_end,:] += max_mask * output_gradient[:,  h, w,:, np.newaxis, np.newaxis]
        
        return input_gradient


In [ ]:
inputs=np.random.randint(1,10, (100,200, 200,32))

In [ ]:
pool = MaxPool2D()
out=pool.forward(inputs)
pool.use_device=True
out1= pool.forward(inputs)

time= 0.989051342010498
time= 0.6579642295837402


In [ ]:
np.sum( np.abs(out - out1))

0.0

# Dense

In [ ]:
A=np.random.randint(1,2, (3,3,2))
B=np.random.randint(1,2, (3,3,2))

Expect:  A*B <=> A[...,0]+B[...,B]

### CPU

In [ ]:
@cuda.jit
def dense_forward_device(inputs,weights,bias,outputs):
  row,col= cuda.grid(2)
  height = weights.shape[0] 
  if(row>=inputs.shape[0] or col >= outputs.shape[1] ): return 
  sum=0
  for i in range(inputs.shape[1]):
    sum+= inputs[row,i] *  weights[i,col]
  outputs[row,col] = sum +bias[0,col]


class Dense(Layer):
    def __init__(self, num_inputs, num_outputs, activation="relu"):
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        np.random.seed(10)
        self.weights = np.random.randn(num_inputs, num_outputs) * np.sqrt(1. / num_inputs)
        np.random.seed(10)
        self.biases = np.zeros((1, num_outputs))
        self.activation= activation
        self.use_device=False 
        self.inputs  =None
    def forward(self, inputs):
        # Save input
        self.inputs = inputs
        
        # Compute output
        start =time.time()
        outputs=np.zeros((inputs.shape[0],self.weights.shape[1]))
        if(self.use_device==False):
          outputs = np.dot(inputs, self.weights)+ self.biases
        else:
          weights_device = cuda.to_device(self.weights)
          biases_device = cuda.to_device(self.biases)
          out_device= cuda.device_array_like(outputs)
          block_size = (32,16)
          grid_size = ((inputs.shape[0]-1)//block_size[0]+1,(inputs.shape[0]-1)//block_size[1]+1)
          inputs_device = cuda.to_device(inputs)
          dense_forward_device[grid_size,block_size](inputs_device,weights_device,biases_device,out_device)
          outputs = out_device.copy_to_host()
        end = time.time()
        print('time=',end-start)

        if(self.activation=="relu"):
          outputs = np.maximum(0,outputs)
        elif self.activation == "softmax":
          outputs = self.softmax( outputs )
        return outputs
    def softmax(self,x):
      e_x = np.exp(x - np.max(x))
      return e_x / e_x.sum(axis=1, keepdims=True)

    def backward(self, output_gradient, learning_rate):
        # Compute input error
        input_error = np.dot(output_gradient, self.weights.T)
        
        # Compute gradients
        weights_gradient = np.dot(self.inputs.T, output_gradient)
        biases_gradient = np.sum(output_gradient, axis=0, keepdims=True)
        
        # Update weights and biases
        self.weights -= learning_rate * weights_gradient
        self.biases -= learning_rate * biases_gradient
        
        return input_error

In [ ]:
inputs=np.random.randint(1,100, (10000,10000))

In [ ]:
dense=Dense(10000,100)
out_host=dense.forward(inputs)
dense.use_device=True
out_device=dense.forward(inputs)
out_host.shape

time= 0.9002511501312256
time= 0.6991419792175293


(10000, 100)

In [ ]:
np.sum(np.abs(out_host-out_device))

7.12350361108875e-08

### GPU

In [ ]:
class Flatten(Layer):
  def __init__(self):
    pass
  def forward(self,inputs):
    self.inputs=inputs
    return inputs.reshape(inputs.shape[0], -1)
  def backward(self,output_gradient,learning_rate):
    shape= self.inputs.shape
    return output_gradient.reshape(shape)

# CNN Model

In [ ]:
class CNNModel:
  def __init__(self, layers:list[Layer]=[]):
    self.layers:list[Layer]=layers
    pass
  def add(self,layer:Layer):
    self.layers.append(layer)
  def forward(self,X):
    output= X
    for layer in self.layers:
            output=layer.forward(output)
    return output

  def fit(self,X_train,Y_train,epochs=1, batch_size=32, learning_rate=0.001, loss="CrossEntropy"):
    num_batch=(len(X_train)-1)/batch_size+1
    for i_epoch in range(epochs):
      train_loss =0
      for i in range(num_batch-1):
        batch_start = i * batch_size
        batch_end = (i + 1) * batch_size
        batch_X= X_train[batch_start: batch_end]
        batch_Y= Y_train[batch_start: batch_end]
        predictions=self.forward(batch_X)
        train_loss += np.sum((predictions - batch_Y) ** 2)

        d_L_d_predictions = 2.0 * (predictions - batch_Y)
      # loss= y_train-y_predict
      print(loss)
      # for layer in self.layers:
      #   Xinputs=layer.back(Xinputs)
  def predict(self,X):
    return self.forward(X)

  def use_device(self, value):
    for layer in self.layers:
      output=layer.use_device=value
  @staticmethod
  def cross_entropy_loss(predictions, targets):
      num_samples = predictions.shape[0]
      softmax = np.exp(predictions - np.max(predictions, axis=1, keepdims=True))
      softmax /= np.sum(softmax, axis=1, keepdims=True)
      log_likelihood = -np.log(softmax[range(num_samples), targets])
      loss = np.sum(log_likelihood) / num_samples
      return loss

  @staticmethod
  def cross_entropy_loss_derivative(predictions, targets):
      num_samples = predictions.shape[0]
      softmax = np.exp(predictions - np.max(predictions, axis=1, keepdims=True))
      softmax /= np.sum(softmax, axis=1, keepdims=True)
      softmax[range(num_samples), targets] -= 1
      softmax /= num_samples
      return softmax

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
model=CNNModel([ 
    Convolution(n_filters=32, filter_size=3, stride=1,activation='relu'),
    Convolution(n_filters=64, filter_size=3, stride=1,activation='relu'),
    MaxPool2D(pool_size=2),
    Flatten(),
    Dense(9216,128, activation='relu'),
    Dense(128,10, activation='softmax')
])
model.use_device(True)


In [ ]:
inputs=np.random.randint(1,100, (100,28,28,1))

In [ ]:
%%time
y=model.predict(inputs)

time: 0.0701451301574707
time: 0.1884913444519043
time= 0.014975786209106445
time= 0.02480483055114746
time= 0.001131296157836914
CPU times: user 285 ms, sys: 22.8 ms, total: 308 ms
Wall time: 307 ms


In [ ]:
y[0].sum()

1.0